In [1]:
import paddlehub as hub
import pandas as pd
import imageio
import numpy as np

C:\Users\Adydio\anaconda\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
def color_func(word,/,font_size,position,random_state,**kwargs):
    if position[0]<500:
        r=random_state.randint(0,40)
    else:
        r=random_state.randint(100,150)
    if position[1]<500:
        g=random_state.randint(0,40)
    else:
        g=random_state.randint(100,150)
    if font_size<50:
        b=random_state.randint(0,40)
    else:
        b=random_state.randint(100,150)
    return (r,g,b)

In [3]:
def transform_mask(imagename):
    mask = np.array(Image.open(f"./{imagename}.png")) 
    mask = mask[:,:,0] # transform 3d image to 2d for easier visualization
    def transform_format(val):
        if val.any()== 0:
            return 255
        else:
            return val
    transformed_mask = np.ndarray((aatrox_mask.shape[0],aatrox_mask.shape[1]), np.int32)
    for i in range(len(aatrox_mask)):
        transformed_mask[i] = list(map(transform_format,aatrox_mask[i]))
    return transformed_mask

In [4]:
def get_image_colors(imagename):
    mask = np.array(Image.open(f"./{imagename}.png"))
    image_colors  = ImageColorGenerator(mask)
    return image_colors

In [5]:
import re
cop = re.compile("[^\u4e00-\u9fa5^a-z^A-Z^0-9]")

In [7]:
mask = imageio.imread("wyy1.png")

In [31]:
df = pd.read_csv("all_comments_3.csv")['content']

In [32]:
test_text = df.tolist()
for i in range(len(test_text)):
    test_text[i] = str(test_text[i])
    test_text[i] = cop.sub('', test_text[i])

In [33]:
lac = hub.Module(name="lac") # 加载模型

In [34]:
results = lac.cut(text=test_text, use_gpu=False, batch_size=1, return_tag=True)

In [35]:
result_word_list = []
for result in results:
    result_word_list.extend(result["word"])

In [36]:
# 载入停用词数据
with open("stopwords.txt","r",encoding="utf-8") as f:
    stop_word_list = [s.strip() for s in f.readlines()]

In [37]:
# 统计词频
word_cloud_dict = {}
for w in result_word_list:
    if w in stop_word_list:
        continue
    if w in word_cloud_dict.keys():
        word_cloud_dict[w] = word_cloud_dict[w] + 1
    else:
        word_cloud_dict[w] = 1

In [38]:
# 转换为数据框
df = pd.DataFrame(pd.Series(word_cloud_dict), columns=['value'])
df = df.reset_index().rename(columns={'index': 'key'})
df.to_csv("word_freq_3.csv",encoding="utf-8-sig")

In [39]:
result_word_list
tex = ""
for word in result_word_list:
    tex = tex + word + " "

In [16]:
import pyecharts.options as opts
from pyecharts.charts import WordCloud
import wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

颜色选项参考网站：https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html

In [41]:
#color_func=color_func
font = r'C:\Users\Adydio\Desktop\msyh.ttf'
w = wordcloud.WordCloud(mask=mask, background_color='white',font_step=4, stopwords=stop_word_list,colormap="winter", font_path=font, max_words=200,min_font_size=6,max_font_size=100)
w.generate(tex)
w.to_file('houqi.png')